In [ ]:
import gym
from dqn import DQNAgent
from network import CNN

In [ ]:
env = gym.make('Breakout-v4')

In [ ]:
q_net = CNN(env.action_space.n)
agent = DQNAgent(env.action_space.n)
SKIP_FRAMES = 4
NUM_EPISODES = 5000
frames_seen = 0
frames = 0
eps_rewards = dict()
total_reward = 0.0
for i in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    eps_reward = 0
    reward, done = 0, False
    while not done:    
        action, eps = agent.policy(observation, reward, done)
        frames_seen += 1
        for _ in range(SKIP_FRAMES):
            frames += 1
            observation, reward, done, _ = env.step(action)
            eps_reward += reward
            if done:
                break
    eps_rewards[i] = eps_reward
    total_reward += eps_reward
    print(f"Episode {i} Epsilon = {eps} Reward = {eps_reward} Avg Reward = {total_reward/(i+1)} Frames seen {frames_seen} Frames {frames}")